In [164]:
import pandas as pd
import numpy as np
import scipy.stats

In [152]:
def generate_experiment_df(size=10000):
    lower = 1
    upper = 60*24
    mu = 30
    sigma = 15
    a = (lower-mu)/sigma
    b = (upper-mu)/sigma
    df = pd.DataFrame(
        data = {'player_id' : np.arange(size).astype(str),
               'experiment' : 0,
               'treatment_id' : np.random.choice(['A','B'],size=size),
                'metric_id' : 'session_length',
                'metric_value' : scipy.stats.truncnorm.rvs(a=a, b=b, loc=mu,scale=sigma,size=size)
               },)
    return df[['player_id','experiment','treatment_id','metric_id','metric_value']]
    

In [153]:
df = generate_experiment_df()
df.head(5)

,player_id,experiment,treatment_id,metric_id,metric_value
0,0,0,A,session_length,58.187713
1,1,0,A,session_length,23.293527
2,2,0,B,session_length,39.685642
3,3,0,B,session_length,16.775225
4,4,0,A,session_length,21.284667


In [154]:
df.groupby(by=['experiment','treatment_id','metric_id']).describe()

metric_value                        \
                                              count       mean        std   
experiment treatment_id metric_id                                           
0          A            session_length       4988.0  30.769587  13.853980   
           B            session_length       5012.0  30.681265  14.000525   

                                                                        \
                                             min        25%        50%   
experiment treatment_id metric_id                                        
0          A            session_length  1.044831  21.031187  30.058082   
           B            session_length  1.058610  20.855239  30.166814   

                                                              
                                              75%        max  
experiment treatment_id metric_id                             
0          A            session_length  40.202625  81.123536  
           B            session_length  40.011888  82.408136

In [155]:
from statsmodels.stats.api import ttest_ind

In [156]:
A = df.query('treatment_id == "A" and metric_id == "session_length"')['metric_value']
B = df.query('treatment_id == "B" and metric_id == "session_length"')['metric_value']

In [157]:
tstat,pvalue,degreeofFreedom = ttest_ind(A,B,alternative='smaller')
print('tstat :'+str(tstat))
print('pvalue :'+str(pvalue))
print('degreeofFreedom :'+str(degreeofFreedom))

tstat :0.31707466905845966
pvalue :0.6244032171573435
degreeofFreedom :9998.0


In [158]:
# Student t-distribution Percent Point Function
from scipy.stats import t
# define probability
p = 0.95
dfr = 9998
# retrieve value <= probability
value = t.ppf(p, dfr)
print(value)
# confirm with cdf
p = t.cdf(value, dfr)
print(p)

1.6450060485564049
0.95


In [159]:
if (-1*value) < tstat:
    print("We do not reject null")
else:
    print("We reject null")

We do not reject null


-1.64 < .20 thus we do not reject the null hypothesis.

#### An alternative to doing this test is to do linear Regression / logistic regression and see the t stat and significance level.

In [160]:
import statsmodels.api as sm

In [161]:
dummy = pd.get_dummies(df['treatment_id'])

In [162]:
df1 = pd.merge(df,dummy,on=df.index)
X  = df1[['A']].values.tolist()
Y  = df1[['metric_value']].values.tolist()
X  = sm.add_constant(X)

In [163]:
model = sm.OLS(Y,X)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1005
Date:                Wed, 03 Apr 2019   Prob (F-statistic):              0.751
Time:                        21:44:47   Log-Likelihood:                -40527.
No. Observations:               10000   AIC:                         8.106e+04
Df Residuals:                    9998   BIC:                         8.107e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         30.6813      0.197    155.956      0.0

In [165]:
# Trying Multivariate Testing

def generate_experiment_df(size=10000):
    lower = 1
    upper = 60*24
    mu = 30
    sigma = 15
    a = (lower-mu)/sigma
    b = (upper-mu)/sigma
    df = pd.DataFrame(
        data = {'player_id' : np.arange(size).astype(str),
               'experiment' : 0,
               'treatment_id' : np.random.choice(['A','B','C'],size=size),
                'metric_id' : 'session_length',
                'metric_value' : scipy.stats.truncnorm.rvs(a=a, b=b, loc=mu,scale=sigma,size=size)
               },)
    return df[['player_id','experiment','treatment_id','metric_id','metric_value']]
    

In [166]:
multiDf = generate_experiment_df()
multiDf.head(5)

,player_id,experiment,treatment_id,metric_id,metric_value
0,0,0,B,session_length,30.164051
1,1,0,B,session_length,46.349394
2,2,0,A,session_length,30.289689
3,3,0,B,session_length,28.549957
4,4,0,C,session_length,17.951847


In [169]:
dummy = pd.get_dummies(multiDf['treatment_id'])
multiDf1 = pd.merge(multiDf,dummy,on=df.index)
X  = multiDf1[['A','B']].values.tolist()
Y  = multiDf1[['metric_value']].values.tolist()
X  = sm.add_constant(X)

In [170]:
model = sm.OLS(Y,X)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.102
Date:                Wed, 03 Apr 2019   Prob (F-statistic):              0.332
Time:                        21:52:09   Log-Likelihood:                -40435.
No. Observations:               10000   AIC:                         8.088e+04
Df Residuals:                    9997   BIC:                         8.090e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         30.5755      0.238    128.266      0.0